# Azure AI Search with Azure AI Vision multimodal embeddings for text-to-image queries

As a scenario, this code shows you an approach for text-to-image vector queries. As a technical sample, it demonstrates how to call a custom embedding model for situations where you want models other an Azure OpenAI or OpenAI for vectorization. The multimodal embeddings used in this sample are provided by [Azure AI Vision 4.0](https://learn.microsoft.com/azure/ai-services/computer-vision/how-to/image-retrieval) and the [Image Retrieval REST API](https://learn.microsoft.com/rest/api/computervision/image-retrieval) which supports built-in vectorization of images. 

For indexing, the pattern uses a custom skill to wrap an Azure function app used to call the Image Retrieval API. Provisioning of this function app and custom skill is fully automated and included as a step in this notebook.

The function app is also used during queries, as the vectorizer. A vectorizer specifies which embedding model to use for vectorizing a text query string. As always, it's strongly recommended that query vectorization is performed using the same embedding model used for document vectorization during indexing.

### Prerequisites

+ [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal), any region and tier, but we recommend Basic or higher for this workload.

+ [Azure Blob storage](https://learn.microsoft.com/azure/storage/common/storage-account-create), used as the data source during indexing.

+ [azd](https://learn.microsoft.com/azure/developer/azure-developer-cli/install-azd), used to deploy an Azure function app and Azure AI Vision in your Azure subscription.

We use the [Azure Python SDK](https://learn.microsoft.com/en-us/python/api/azure-search-documents/?view=azure-python-preview) for indexer-driven indexing and vector query operations.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

In [ ]:
! pip install -r azure-search-vector-image-python-sample-requirements.txt

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv() # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
blob_container_name = os.environ["BLOB_CONTAINER_NAME"]
function_app_url = os.environ["function_app_url"]
sas_token = os.environ["SAS_TOKEN"]

In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import SearchIndexerDataContainer, SearchIndexerDataSourceConnection
# Create a data source 
ds_client = SearchIndexerClient(endpoint, credential)
container = SearchIndexerDataContainer(name=blob_container_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=blob_connection_string,
    container=container
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")